In [30]:
import hopsworks
import os
from datetime import datetime

In [2]:
project = hopsworks.login()
mr = project.get_model_registry()

2024-10-17 15:44:00,772 INFO: Python Engine initialized.

Logged in to project, explore it here https://demo.hops.works/p/123


In [3]:
# get model object
model = mr.get_model("xgboost_fraud_model", version=1)

In [138]:
%%writefile /tmp/predict_example.py
import os
import numpy as np
import pandas as pd
import hopsworks
import joblib
from datetime import datetime

class Predict(object):

    def __init__(self, model):
        # initialize serving
        project = hopsworks.login()
        fs = project.get_feature_store()
        
        # Initialize the feature view using the model provenance
        model_fv = model.get_feature_view()
        self.fraud_fv = fs.get_feature_view(model_fv.name, model_fv.version)

        # load the trained model
        self.model = joblib.load(
            os.environ["ARTIFACT_FILES_PATH"] + "/xgboost_fraud_model.pkl"
        )

        print("Initialization Complete")

    def predict(self, inputs):
        feature_vector = self.prepare_feature_vector(inputs[0])
        return self.model.predict(np.asarray(feature_vector).reshape(1, -1)).tolist()

    def prepare_feature_vector(self, input_transaction):
        # Fetch Inference Helper feature values to compute on-demand features
        helpers_df = self.fraud_fv.get_inference_helper(
            entry={"account_id": input_transaction["account_id"]},
            return_type="dict"
        )
        helpers_df['latitude'] = input_transaction['latitude']
        helpers_df['longitude'] = input_transaction['longitude']
        helpers_df["datetime"] = datetime.strptime(input_transaction['datetime'], "%Y-%m-%d %H:%M:%S")
        helpers_df['transaction_city'] = input_transaction['city']
        
        passed_features = {
            "amount": input_transaction["amount"],
            "category": input_transaction["category"]
        }
        
        feature_vector = self.fraud_fv.get_feature_vector(
            {"account_id": input_transaction["account_id"]}, 
            allow_missing=True,
            passed_features=passed_features,
            request_parameters=helpers_df,
            transform=True,
            return_type="pandas"
        )     

        return feature_vector

Overwriting /tmp/predict_example.py


In [139]:
# Get the dataset API for the current project
dataset_api = project.get_dataset_api()

# Specify the local file path of the Python script to be uploaded
local_script_path = "/tmp/predict_example.py"

# Upload the Python script to the "Models", and overwrite if it already exists
uploaded_file_path = dataset_api.upload(local_script_path, "Models", overwrite=True)

# Create the full path to the uploaded script for future reference
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/1952 elapsed<00:00 remaining<?

In [140]:
# Deploy the fraud model
deployment = model.deploy(
    name="fraudonline",
    script_file=predictor_script_path,
)

Deployment created, explore it at https://demo.hops.works/p/123/deployments/1044
Before making predictions, start the deployment by using `.start()`


In [141]:
deployment.start(await_running=300)

  0%|          | 0/5 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [142]:
# Print the name of the deployment
print("Deployment: " + deployment.name)

# Display information about the deployment
deployment.describe()

Deployment: fraudonline
{
    "api_protocol": "REST",
    "artifact_version": 4,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2024-10-17T17:01:00.687Z",
    "creator": "Fabio Buso",
    "description": null,
    "id": 1044,
    "inference_logging": "NONE",
    "model_framework": "PYTHON",
    "model_name": "xgboost_fraud_model",
    "model_path": "/Projects/payment_fraud/Models/xgboost_fraud_model",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "fraudonline",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 2,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "memory": 32
        }
    },
    "project_namespace": "payment-fraud",
    "requested_instances": 1,
    "serving_tool": "KSERVE"
}


In [147]:
# Make a prediction
deployment.predict(inputs = [{
    'account_id': 'b9ae1b2174c5333c3a2c8bab6d64c3b6',
    'amount': 1234,
    'category': 'Cash Withdrawal',
    'city': 'Collinwood',
    'latitude': 40.65538,
    'longitude': -74.38987,
    'datetime': '2024-04-19 13:31:37',
}])

{'predictions': [0]}